dense embbeding
 - 모든 차원에 값이 존재(dense = 촘촘함)
 - 모든 차원이 항상 값을 가짐  [0.13, -0.27, 0.88, ..., 0.01] 
 - 의미 기반 표현 (단어, 문장, 문선의 의미를 압축)
 
Sparse embedding
 - 대부분의 값이 0(sparse = 드문드문함)
 - 단어 기반(lexical)
 

dense embedding은 "텍스트의 의미를 압축해서 표현"   
sparse embedding은 "텍스트 내 어떤 단어가 포함되었는지 중심으로 표현"

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceEmbeddings(

    
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [41]:
#샘플데이터터

from langchain_core.documents import Document

texts = [
    "안녕하세요. 제 이름은 김용현입니다. lanchaing에서 임베딩을 공부중에 있습니다."
]




In [4]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")


Model: 		BAAI/bge-m3
Dimension: 	1024


dense vector

In [42]:
from FlagEmbedding import BGEM3FlagModel

bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  
bge_encoded = bge_flagmodel.encode(texts, return_dense=True) #일반적인 임베딩


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [43]:
bge_encoded

{'dense_vecs': array([[-0.0510617 , -0.00911605, -0.03279521, ..., -0.03671136,
          0.03529508,  0.00833996]], dtype=float32),
 'lexical_weights': None,
 'colbert_vecs': None}

sparse embedding

In [48]:
bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  
bge_encoded_sparse = bge_flagmodel.encode(texts, return_dense=False, return_sparse=True)


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [47]:
bge_encoded_sparse

{'dense_vecs': array([[-0.0510617 , -0.00911605, -0.03279521, ..., -0.03671136,
          0.03529508,  0.00833996]], dtype=float32),
 'lexical_weights': [defaultdict(int,
              {'107687': 0.225349,
               '5': 0.05269464,
               '6600': 0.12692782,
               '50932': 0.1826216,
               '697': 0.12572032,
               '8237': 0.21062742,
               '5358': 0.25677747,
               '14020': 0.27770653,
               '5826': 0.14465418,
               '2515': 0.10470968,
               '1436': 0.17326719,
               '214': 0.16522749,
               '1180': 0.08300017,
               '20945': 0.11303122,
               '42431': 0.2170032,
               '50260': 0.162927,
               '413': 0.015961185,
               '83773': 0.17251246,
               '7094': 0.11248326,
               '480': 0.04173658,
               '3292': 0.073996305})],
 'colbert_vecs': None}

In [51]:
from transformers import AutoTokenizer

# BGE-M3 모델과 동일한 tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# 예시: sparse vector 키 중 하나
token_id = 14020  # 예: sparse vector에서 나온 키

# 단일 토큰 디코딩
token_str = tokenizer.decode([token_id])
print(f"{token_id} → '{token_str}'")


14020 → '현'


단어의 가중치의 기준 (bge-m3)
 1. Term Frequency (TF)   
    해당 문장에서 단어가 얼마나 자주 등장했는가
 2. Inverse Document Frequency (IDF)   
    그 단어가 전체 코퍼스에서 얼마나 희귀한가

- 흔한 단어(은,는,이,가)는 낮은 가중치, 문서에서만 등장하는 특이한 단어는 높은 가중치


차이

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


texts = ["오늘 날씨 참 좋네요", "오늘 기분 정말 좋아요"]

# dense-only 인코딩
dense_encoded = bge_flagmodel.encode(texts, return_dense=True, return_sparse=False)

# sparse-only 인코딩
sparse_encoded = bge_flagmodel.encode(texts, return_dense=False, return_sparse=True)

# 1. DENSE 유사도 계산 (cosine similarity)
dense_vecs = dense_encoded["dense_vecs"]
dense_sim = cosine_similarity([dense_vecs[0]], [dense_vecs[1]])[0][0]

# 2. SPARSE 유사도 계산 (lexical_weights 기반 단순 dot product)
# sparse 벡터는 dict 형태이므로 numpy vector로 변환
def sparse_dict_to_vector(sparse_dict, vocab):
    vec = np.zeros(len(vocab))
    for i, token in enumerate(vocab):
        vec[i] = sparse_dict.get(token, 0)
    return vec

vocab = list(set(sparse_encoded["lexical_weights"][0].keys()) | 
             set(sparse_encoded["lexical_weights"][1].keys()))

v1 = sparse_dict_to_vector(sparse_encoded["lexical_weights"][0], vocab)
v2 = sparse_dict_to_vector(sparse_encoded["lexical_weights"][1], vocab)

sparse_sim = cosine_similarity([v1], [v2])[0][0]

# 결과 출력
print(f"🟦 Dense 유사도 (의미): {dense_sim:.4f}")
print(f"🟧 Sparse 유사도 (단어 일치): {sparse_sim:.4f}")

🟦 Dense 유사도 (의미): 0.8880
🟧 Sparse 유사도 (단어 일치): 0.2839


In [18]:
dense_vecs

array([[-0.03337294,  0.02527683, -0.00537117, ..., -0.03823467,
        -0.01806252,  0.01654678],
       [ 0.00020291,  0.01154577, -0.00948457, ..., -0.03402546,
        -0.01071138,  0.04857172]], dtype=float32)

In [19]:

texts = ["저 영화배우는 정말 못 생겼더라", "저 영화배우는 못 하는게 없더라"]

# dense-only 인코딩
dense_encoded = bge_flagmodel.encode(texts, return_dense=True, return_sparse=False)

# sparse-only 인코딩
sparse_encoded = bge_flagmodel.encode(texts, return_dense=False, return_sparse=True)

# 1. DENSE 유사도 계산 (cosine similarity)
dense_vecs = dense_encoded["dense_vecs"]
dense_sim = cosine_similarity([dense_vecs[0]], [dense_vecs[1]])[0][0]

# 2. SPARSE 유사도 계산 (lexical_weights 기반 단순 dot product)
# sparse 벡터는 dict 형태이므로 numpy vector로 변환
def sparse_dict_to_vector(sparse_dict, vocab):
    vec = np.zeros(len(vocab))
    for i, token in enumerate(vocab):
        vec[i] = sparse_dict.get(token, 0)
    return vec

vocab = list(set(sparse_encoded["lexical_weights"][0].keys()) | 
             set(sparse_encoded["lexical_weights"][1].keys()))

v1 = sparse_dict_to_vector(sparse_encoded["lexical_weights"][0], vocab)
v2 = sparse_dict_to_vector(sparse_encoded["lexical_weights"][1], vocab)

sparse_sim = cosine_similarity([v1], [v2])[0][0]

# 결과 출력
print(f"🟦 Dense 유사도 (의미): {dense_sim:.4f}")
print(f"🟧 Sparse 유사도 (단어 일치): {sparse_sim:.4f}")

🟦 Dense 유사도 (의미): 0.7739
🟧 Sparse 유사도 (단어 일치): 0.6959


In [ ]:
texts = ["아빠 가방에 들어갔습니다.", "아빠가 방에 들어갔습니다."]

# dense-only 인코딩
dense_encoded = bge_flagmodel.encode(texts, return_dense=True, return_sparse=False)

# sparse-only 인코딩
sparse_encoded = bge_flagmodel.encode(texts, return_dense=False, return_sparse=True)

# 1. DENSE 유사도 계산 (cosine similarity)
dense_vecs = dense_encoded["dense_vecs"]
dense_sim = cosine_similarity([dense_vecs[0]], [dense_vecs[1]])[0][0]

# 2. SPARSE 유사도 계산 (lexical_weights 기반 단순 dot product)
# sparse 벡터는 dict 형태이므로 numpy vector로 변환
def sparse_dict_to_vector(sparse_dict, vocab):
    vec = np.zeros(len(vocab))
    for i, token in enumerate(vocab):
        vec[i] = sparse_dict.get(token, 0)
    return vec

vocab = list(set(sparse_encoded["lexical_weights"][0].keys()) | 
             set(sparse_encoded["lexical_weights"][1].keys()))

v1 = sparse_dict_to_vector(sparse_encoded["lexical_weights"][0], vocab)
v2 = sparse_dict_to_vector(sparse_encoded["lexical_weights"][1], vocab)

sparse_sim = cosine_similarity([v1], [v2])[0][0]

# 결과 출력
print(f"🟦 Dense 유사도 (의미): {dense_sim:.4f}")
print(f"🟧 Sparse 유사도 (단어 일치): {sparse_sim:.4f}")

🟦 Dense 유사도 (의미): 0.7900
🟧 Sparse 유사도 (단어 일치): 0.6786


In [57]:
texts = ["내가 제일 존경하는 인물은 이순신 정군입니다.", "이순신 장군은 정말 위대합니다."]

# dense-only 인코딩
dense_encoded = bge_flagmodel.encode(texts, return_dense=True, return_sparse=False)

# sparse-only 인코딩
sparse_encoded = bge_flagmodel.encode(texts, return_dense=False, return_sparse=True)

# 1. DENSE 유사도 계산 (cosine similarity)
dense_vecs = dense_encoded["dense_vecs"]
dense_sim = cosine_similarity([dense_vecs[0]], [dense_vecs[1]])[0][0]

# 2. SPARSE 유사도 계산 (lexical_weights 기반 단순 dot product)
# sparse 벡터는 dict 형태이므로 numpy vector로 변환
def sparse_dict_to_vector(sparse_dict, vocab):
    vec = np.zeros(len(vocab))
    for i, token in enumerate(vocab):
        vec[i] = sparse_dict.get(token, 0)
    return vec

vocab = list(set(sparse_encoded["lexical_weights"][0].keys()) | 
             set(sparse_encoded["lexical_weights"][1].keys()))

v1 = sparse_dict_to_vector(sparse_encoded["lexical_weights"][0], vocab)
v2 = sparse_dict_to_vector(sparse_encoded["lexical_weights"][1], vocab)

sparse_sim = cosine_similarity([v1], [v2])[0][0]

# 결과 출력
print(f"🟦 Dense 유사도 (의미): {dense_sim:.4f}")
print(f"🟧 Sparse 유사도 (단어 일치): {sparse_sim:.4f}")

🟦 Dense 유사도 (의미): 0.7582
🟧 Sparse 유사도 (단어 일치): 0.4282


sparse 임베딩은 단어가 정확히 포함되어 있는냐를 보기때문에, 작은 오타나 표현차이도 유사도를 크게 떨어뜨릴 수 있음.   
이 점은 dens embedding과 크게 다른 특징
